<a href="https://colab.research.google.com/github/Meenakshi2434/U-2-Net-Computer-Vision-Project/blob/main/Inference_script_task1_pth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import cv2

# Define the data transforms (same as the training script)
image_transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the trained model (assuming you have a saved model file)
model = U2Net()
model.load_state_dict(torch.load('path/to/your/model.pth'))  # Replace with actual path
model.eval()  # Set the model to evaluation mode

# Define a function to predict mask for a single image
def predict_mask(image_path):
  # Load the image
  image = Image.open(image_path)

  # Preprocess the image
  image = image_transform(image)

  # Add a batch dimension
  image = image.unsqueeze(0)

  # Move data to device (CPU or GPU)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  image = image.to(device)

  # Predict the mask
  with torch.no_grad():
    output = model(image)

  # Process the predicted mask (e.g., thresholding, removing batch dimension)
  predicted_mask = output.squeeze(0).cpu()  # Remove batch dimension and move to CPU
  predicted_mask = torch.sigmoid(predicted_mask)  # Apply sigmoid activation for probability
  predicted_mask = (predicted_mask > 0.5).float()  # Threshold to get binary mask

  return predicted_mask

# Example usage
image_path = 'path/to/your/image.jpg'  # Replace with your image path
predicted_mask = predict_mask(image_path)

# You can further process the predicted mask (e.g., convert to NumPy array, visualize)
print(f"Predicted mask shape: {predicted_mask.shape}")